### **Postavka Julie u Google Colab**

Potrebno je preuzeti ovaj notebook, upload-ovati ga i pokrenuti kôd ispod prilikom kreiranja notebook-a za sve vježbe. Nakon pokretanja, potrebno je restartovati notebook. U *Runtime* kliknite na *Change runtime type* i odaberite Juliu i GPU. Sada možete koristiti Juliu za sve što radite.

U slučaju da vam krene izbacivati greške radi gubitka trenutne sesije izvršite sve naredbe opet i restartujte notebook.




In [ ]:
!curl -sSL "https://julialang-s3.julialang.org/bin/linux/x64/1.5/julia-1.5.2-linux-x86_64.tar.gz" -o julia.tar.gz
!tar -xzf julia.tar.gz -C /usr --strip-components 1
!rm -rf julia.tar.gz*
!julia -e 'using Pkg; pkg"add IJulia; precompile"'
!echo "DONE"

 Installing known registries into `~/.julia`
######################################################################## 100.0%
      Added registry `General` to `~/.julia/registries/General`
  Resolving package versions...
  Installed ZeroMQ_jll ───────── v4.3.2+6
  Installed SoftGlobalScope ──── v1.1.0
  Installed ZMQ ──────────────── v1.2.1
  Installed Zlib_jll ─────────── v1.2.11+18
  Installed MbedTLS ──────────── v1.0.3
  Installed IJulia ───────────── v1.23.2
  Installed TOML ─────────────── v1.0.3
  Installed ArgTools ─────────── v1.1.1
  Installed Conda ────────────── v1.6.0
  Installed Preferences ──────── v1.2.2
  Installed MbedTLS_jll ──────── v2.16.8+1
  Installed MozillaCACerts_jll ─ v2021.1.19+0
  Installed VersionParsing ───── v1.2.1
  Installed Parsers ──────────── v2.1.2
  Installed LibSSH2_jll ──────── v1.9.0+3
  Installed JLLWrappers ──────── v1.3.0
  Installed libsodium_jll ────── v1.0.19+0
  Installed LibCURL_jll ──────── v7.70.0+2
  Installed NetworkOptions ───── v1

In [ ]:
using LinearAlgebra

In [ ]:
function formirajSimpleksTabelu(A, b, c)
broj_kolona = size(A,2)
j_matrica = I(size(b,2))+zeros(size(b,2),size(b,2))
simplex_tabela= hcat(vcat(hcat(A, j_matrica),hcat(c,zeros(1, size(j_matrica,2)))), vcat(transpose(b),[0]))
return simplex_tabela
end

formirajSimpleksTabelu (generic function with 1 method)

Ako u c vektoru nema pozitivnih elemenata, algoritam je dosao do kraja.

In [ ]:
function jesunegativni(A)
  for i=1:size(A,2)
      if A[1,i] > 0
        return 0
      end
  end
  return 1
end

jesunegativni (generic function with 1 method)

In [ ]:
function nadiVodecuKolonu(A)
        pozicija=1
        max=0
        for i=1:size(A,2)
          if A[1,i]>0 && A[1,i] > max
            max=A[1,i]
            pozicija=i
          end
        end
        return pozicija
end


nadiVodecuKolonu (generic function with 1 method)

In [ ]:
function nadiVodeciRed(A,B, indeks)
    min=B[1,1]/A[1,indeks]
    pozicija=1
    for i=1:(size(A,1)-1)
    if(A[i,indeks] <=0)
      return 0
    end
    if(A[i,indeks] <=0)
      return 0
      end
      if A[i, indeks] > 0 && B[1,i]/A[i,indeks]<min
        min=B[1,i]/A[i,indeks]
        pozicija=i
      end
    end
    return pozicija
end

nadiVodeciRed (generic function with 1 method)

Ako u vodecoj koloni nema pozitivnih elemenata, rjesenje je u beskonacnosti.

In [ ]:

function rijesi_simplex(A, b, c)
          tabela=formirajSimpleksTabelu(A,b,c)
          baza = vcat([size(A,2)+1:size(A,2)+size(A,1);],[0])
          v_kolona=nadiVodecuKolonu(c)
          v_red=nadiVodeciRed(tabela, b, v_kolona)
          if(v_red == 0)
          return "Beskonacno"
          end
          br_kolona=size(tabela,2)
         while jesunegativni(transpose(tabela[size(tabela,1),:]))!=1
         pivot=tabela[v_red, v_kolona]
           for i=1:br_kolona
           tabela[v_red, i]=tabela[v_red, i]/pivot
           end
           for i=1:size(tabela,1)
              pom = tabela[i,v_kolona]
              for j=1:size(tabela,2)
                if j == v_kolona && tabela[i,v_kolona] <=0
                  return 0
                end 
                  if i != v_red
                    tabela[i,j]=tabela[i,j]-pom*tabela[v_red,j]
                  end
              end
            end
              baza[v_red,1]=v_kolona
              if jesunegativni(transpose(tabela[size(tabela,1),:]))!=1
              v_kolona=nadiVodecuKolonu(tabela[size(tabela,1),:])
              v_red=nadiVodeciRed(tabela, transpose(tabela[:,size(tabela,2)]), v_kolona) 
              if(v_red == 0)
                return "Beskonacno"
              end
              end
        end
               rjesenje = hcat(baza, tabela[:,size(tabela,2)])
              rjesenje[size(rjesenje,1), size(rjesenje,2)]*=(-1)
          return rjesenje
end



rijesi_simplex (generic function with 1 method)

\n

\n

\n

In [ ]:
rijesi_simplex([0.5 0.3; 0.1 0.2], [150 60], [3 1])

3×2 Array{Float64,2}:
 1.0  300.0
 4.0   30.0
 0.0  900.0

In [ ]:
rijesi_simplex([6 9; 2 1], [100 20], [2000 3000])

3×2 Array{Float64,2}:
 2.0     11.1111
 4.0      8.88889
 0.0  33333.3

Prva kolona označava indekse varijabli koje ulaze u bazu, dok druga kolona predstavlja vrijendosti tih elemenata, osim sto zadnji element u prvoj koloni oznacava vrijednost Z. 

In [ ]:
rijesi_simplex([2.5 2.5 2.6; 1 1 1], [11.55 25.7], [17.5 15 13])

3×2 Array{Float64,2}:
 1.0   4.62
 5.0  21.08
 0.0  80.85

In [ ]:
function VodeciRed(A,B)
  min=-1
  for i = 1:size(B,1)-2
      if A[i,1]/B[i,1]>0 
          min=A[i,1]/B[i,1]
          break
        end
    end
    if min==-1 
      return 0
    end
    pozicija=0
    for i = 1:size(A,1)-2
        if B[i,1]>0
          q=A[i,1]/B[i,1]
          if q<=min
            min=q
            pozicija=i
          end
      end
    end
    return pozicija      
end







VodeciRed (generic function with 1 method)

In [ ]:
function VodecaKolona(A, vjestacke) 
  max=0
  pozicija=0
  for i=3:size(A, 2)
    if A[1,i]>0 && A[1,i]>max && !((i-2) in vjestacke)
      max=A[1,i]
      pozicija=i
      end
  end
  return pozicija
end

VodecaKolona (generic function with 1 method)

In [ ]:

function rijesi_simplex(goal,c,A,b,csigns,vsigns)
b = transpose(b);
csigns = transpose(csigns);
vsigns = transpose(vsigns);
if size(A,2)!=size(c,2) || size(A,1)!=size(b,2) || size(c,2)!=size(vsigns,2) || size(b,2)!=size(csigns,2) || (goal!="min" &&  goal!="max")
  return "Greska ulaznih podataka","";
end
baza=zeros(1,size(csigns,2));
M=zeros(1,size(A,2));
brojac=size(c,2);
vjestacke=Vector{Float64}();
vjestackeParovi=zeros(1,size(csigns,2));
dopunskeParovi=zeros(1, size(csigns,2));
pom=c;

# bi nepozitivno:#
for i=1: size(b,1)
  if b[i]<0
    csigns[1,i]=(-1)*csigns[1,i];
    for j=1; size(A, 2)
      A[i,j]=(-1)*A[i,j];
    end
  end
end

parovi=zeros(1,size(c,2));
#ogranicenja nisu >=#
for i=1: size(vsigns, 2)
  if vsigns[1,i]==-1
        c[i]=(-1)*c[i];
        for j=1: size(A, 1)
          A[j,i]=(-1)*A[j,i];
        end
  elseif vsigns[1,i]==0
    c=hcat(c, [(-1)*c[i]]);
    M=hcat(M, [0]);
    parovi[1,i]=size(A,2)+1;
    kolona=A[:,i]
    for j=1: size(kolona,1)
      kolona[j,1]=(-1)*kolona[j,1];
    end
    A=hcat(A,kolona);
    brojac=brojac+1;
  end
end

#Dodavanje dopunskih i vještačkih varijabli#
slobodni=0;
for i=1: size(csigns,2)
  if csigns[1,i]==-1
      #ako je <= dodajemo dopunske:#
      nova=zeros(size(A,1),1);
      nova[i,1]=1;
      A=hcat(A,nova);
      M=hcat(M, [0]);
      c=hcat(c, [0]);
      brojac=brojac+1;
      baza[1,i]=brojac;
      dopunskeParovi[1,i]=size(c,2);
   elseif csigns[i]==0
      #ako je ograničenje = dodajemo vještačku: #
      nova=zeros(size(A,1),1);
      nova[i,1]=1;
      A=hcat(A,nova);
      M=hcat(M, [0]);
      c=hcat(c, [0]);
      if goal=="max"
        for k=1: size(M, 2)
          if k!=(brojac+1)
            M[1,k]=A[1,k]-M[1,k];
          end
        end
        slobodni=slobodni-b[1,i];
      else
       for k=1: size(M, 2)
          if k!=(brojac+1)
              M[1,k]=M[1,k]+A[i,k];
            end
        end
        slobodni=slobodni+b[1,i];
      end
      brojac=brojac+1;
      baza[1,i]=brojac;
      push!(vjestacke, brojac);
      vjestackeParovi[1,i]=size(c,2);
      else 
      #Dodajemo dopunsku: #
        nova=zeros(size(A,1),1);
        nova[i,1]=-1;
        A=hcat(A,nova);
        M=hcat(M, [0]);
        c=hcat(c, [0]);
        brojac=brojac+1;
        dopunskeParovi[1,i]=size(c,2);
        #Dodajemo vještačku#
        nova=zeros(size(A,1),1);
        nova[i,1]=1;
        A=hcat(A,nova);
        M=hcat(M, [0]);
        c=hcat(c, [0]);
        if goal=="max"
            for k=1: size(M, 2)
              if k!=(brojac+1)
                M[1,k]=A[i,k]-M[1,k];
              end
            end
            slobodni=slobodni-b[1,i];
       else
          for k=1: size(M, 2)
              if k!=(brojac+1)
                  M[1,k]=M[1,k]+A[i,k];
              end
          end
          slobodni=slobodni+b[1,i];
      end
      brojac=brojac+1;
      baza[1,i]=brojac;
      push!(vjestacke, brojac);
      vjestackeParovi[1,i]=size(c,2);
    end
end

#u slucaju minimizacije negiramo koeficjente u funkciji cilja#
if goal=="min"
  for i=1: size(c, 2)
    c[1,i]=(-1)*c[1,i];
  end
end

#Formiranje simplex tabele 
A=vcat(A, M);
A=vcat(A,c);
b=hcat(b, [slobodni 0]);
A=hcat(transpose(b),A);
baza=hcat(baza, [0 0]);
A=hcat(transpose(baza),A);
  nadjen=0
 while nadjen==0
  posljednja=A[size(A,1),:];
  pretposljednja=A[size(A,1)-1,:];
  Mz=posljednja
  vjestUBaziNisuNule=false;
  vjestackeUBazi=false;
  for i=1: size(A,1)-2
    if A[i,1] in vjestacke
      vjestackeUBazi=true;
      if A[i,2]!=0.0
        vjestUBaziNisuNule=true;
        break;
      end
    end
  end
  if vjestackeUBazi
    Mz=pretposljednja;
  end
  indeks_vodeca_kolona=VodecaKolona(transpose(Mz), vjestacke)
    if indeks_vodeca_kolona==0 && vjestackeUBazi && vjestUBaziNisuNule
      return "Problem nema rjesenja","";
    elseif indeks_vodeca_kolona==0 
    break
  end
  vodeca_kolona=A[:,indeks_vodeca_kolona]
  b_kolona=A[:,2]
  indeks_vodeci_red=VodeciRed(b_kolona, vodeca_kolona)
  if indeks_vodeci_red==0
     return "rjesenje u besonacnosti","";  
  end
  A[indeks_vodeci_red,1]=indeks_vodeca_kolona-2
  pivot=A[indeks_vodeci_red, indeks_vodeca_kolona]
  #Djeljenje svih elemenata u vodećem redu sa pivotom#
  for i=2:size(A, 2)
      A[indeks_vodeci_red,i]=A[indeks_vodeci_red,i]/pivot
  end

  #Azuriranje tabele#
  for i=1: size(A,1)
      if i!=indeks_vodeci_red
        faktor=A[i,indeks_vodeca_kolona]
          for j=2:size(A,2)
            A[i,j]=A[i,j]-faktor*A[indeks_vodeci_red,j]
          end
      end 
  end
end
#Očitavanje optimalnog rješenja:#
nedegenerirano = true;
 rjesenje=zeros(1,size(A,2)-2)
    for i=1:size(A, 1)-2
      rjesenje[Int(A[i,1])]=A[i,2]
    end
    for i=1:size(A, 1)-2
        if A[i,2] == 0 nedegenerirano = false;
        break;
      end
    end
    Z=A[size(A,1),2]
    if goal=="max"
        Z=Z*(-1);
    end
    #Očitavanje vrijedosti promjenljivih#    
    opt_izvorne_prom=rjesenje[:, 1:size(pom,2)];
    for i=1: size(pom, 2)
        if parovi[1, i]!=0
          opt_izvorne_prom[1,i]=opt_izvorne_prom[1,i]-rjesenje[1, Int(parovi[1,Int(i)])];
        end
    end
    return Z, opt_izvorne_prom
    return true; 
end


rijesi_simplex (generic function with 1 method)

In [ ]:
goal="max"
        c=[3000 2000 1000]
        A=[1 0 2; 2 2 1; 0 1 0]
        b=[1000; 2500; 400]
        csigns=[0;0;0]
        vsigns=[1; 1]
        Z, opt_izvorne_prom = rijesi_simplex(goal,c,A,b,csigns,vsigns)
        print("Z: ")
        println(Z)
        print("X: ")
        println(opt_izvorne_prom)

Z: Greska ulaznih podataka
X: 


In [ ]:
rijesi_simplex("min", [32 56 50 60], [1 1 1 1;250 150 400 200;0 0 0 1;0 1 1 0], [1;300;0.3;0.5],[0;1;-1;-1],[1; 1; 1; 1])

(37.99999999999999, [0.6666666666666666 0.0 0.33333333333333337 0.0])

In [ ]:
goal="min"
        c=[0.36 0.51]
        A=[0.31 0.08;0.52 0.46; 0.59 0.43]
        b=[15.52;4.94;4.94]
        csigns=[-1;0;1]
        vsigns=[1; 1]
        Z, opt_izvorne_prom = rijesi_simplex(goal,c,A,b,csigns,vsigns)
        print("Z: ")
        println(Z)
        print("X: ")
        println(opt_izvorne_prom)

Z: 3.42
X: [9.500000000000002 0.0]


In [ ]:
goal="max"
        c=[40 30]
        A=[3 1.5;1 1;2 1;3 4;1 0;0 1]
        b=[300;80;200;360;60;60]
        csigns=[-1;-1;-1;-1;-1;-1]
        vsigns=[1; 1]
        Z, opt_izvorne_prom = rijesi_simplex(goal,c,A,b,csigns,vsigns)
        print("Z: ")
        println(Z)
        print("X: ")
        println(opt_izvorne_prom)

Z: 3000.0
X: [60.0 20.0]


In [ ]:
goal="min"
c=[0.35 0.53]
A=[0.25 0.15; 0.45 0.55; 0.65 0.45]
b=[6.75;2.09;2.09]
csigns=[-1;0;1]
vsigns=[1;1]
        Z, opt_izvorne_prom = rijesi_simplex(goal,c,A,b,csigns,vsigns)
        print("Z: ")
        println(Z)
        print("X: ")
        println(opt_izvorne_prom)

Z: 1.6255555555555552
X: [4.644444444444444 0.0]
